<a href="https://colab.research.google.com/github/kozltv/pMA_stabilnost/blob/main/pMA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TOC:
* [Part 0: Installation](#first-bullet)
* [Part 1: find Movement onset](#second-bullet)

## Part 0: Installation

__1. Install libraries__

In [ ]:
!jupyter lab --watch

[I 2021-11-22 12:44:22.428 ServerApp] jupyterlab | extension was successfully linked.
[I 2021-11-22 12:44:22.445 ServerApp] Writing notebook server cookie secret to /Users/ksenia.kozlova/Library/Jupyter/runtime/jupyter_cookie_secret
[I 2021-11-22 12:44:22.858 ServerApp] nbclassic | extension was successfully linked.
[I 2021-11-22 12:44:22.907 ServerApp] The port 8888 is already in use, trying another port.
[I 2021-11-22 12:44:22.912 LabApp] JupyterLab extension loaded from /opt/anaconda3/lib/python3.8/site-packages/jupyterlab
[I 2021-11-22 12:44:22.913 LabApp] JupyterLab application directory is /opt/anaconda3/share/jupyter/lab
[I 2021-11-22 12:44:22.913 LabApp] Starting JupyterLab watch mode...
[I 2021-11-22 12:44:29.592 LabApp] > node /opt/anaconda3/lib/python3.8/site-packages/jupyterlab/staging/yarn.js run watch
yarn run v1.21.1
$ webpack --watch
assets by path *.js 51.7 MiB
  assets by chunk 33.6 MiB (id hint: vendors) 47 assets
assets by chunk 2.79 MiB (auxiliary id hint: vendors)

[W 2021-11-22 12:46:31.516 ServerApp] Notebook pMA_v5_auto_part_2.ipynb is not trusted
[I 2021-11-22 12:46:33.984 ServerApp] Kernel started: 7d96986c-83dd-4a8a-97ea-f2d840b3a2f5
[I 2021-11-22 12:48:32.503 ServerApp] Saving file at /pMA_v5_auto_part_2.ipynb
[W 2021-11-22 12:48:32.504 ServerApp] Notebook pMA_v5_auto_part_2.ipynb is not trusted
[I 2021-11-22 12:49:10.994 ServerApp] Saving file at /pMA_v4.ipynb
[W 2021-11-22 12:49:10.995 ServerApp] Notebook pMA_v4.ipynb is not trusted
[I 2021-11-22 12:50:32.653 ServerApp] Saving file at /pMA_v5_auto_part_2.ipynb
[W 2021-11-22 12:50:32.654 ServerApp] Notebook pMA_v5_auto_part_2.ipynb is not trusted
[I 2021-11-22 12:52:32.830 ServerApp] Saving file at /pMA_v5_auto_part_2.ipynb
[W 2021-11-22 12:52:32.831 ServerApp] Notebook pMA_v5_auto_part_2.ipynb is not trusted
[I 2021-11-22 12:54:33.039 ServerApp] Saving file at /pMA_v5_auto_part_2.ipynb
[W 2021-11-22 12:54:33.040 ServerApp] Notebook pMA_v5_auto_part_2.ipynb is not trusted
[I 2021-11-22 12

In [1]:
import numpy as np
import mne
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import warnings
warnings.filterwarnings('ignore')
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gsp
import gspread
from ipywidgets import widgets  
from ipywidgets import interact, interactive, fixed, interact_manual 

#%matplotlib notebook
%matplotlib widgets

KeyError: 'widgets'

__2. Run functions__

In [2]:
# To work with google sheets
#key_path = '/Users/ksenia.kozlova/Documents/GitHub/pma_grant'

def find_movement(data, time, mscl, window, thrs, thrs2, positive):
    '''Find movement

    arguments: 
      raw: DataFrame, filtered emg
      time: list with [start, end], contains start and end of raw data to analise
      mscl: str, name of EMG chanel
      window: int, window size in ms 
      thrs: int, threshold in uV to find movement start in the whole signal
      thrs2:  int, threshold in uV to find more accurate movement start/end moment in the window 
          where movement started/ended based in thrs
      positive: float, percentage of ticks in the window which should be more or eaqual
          to thrs/thrs2
    '''
    
    # 0. Select data
    # ms to select time window to find movement start (ms)
    try:
        start = time[0]
        end=time[1]
    except:
        start = time[0]
        end = raw.to_data_frame().time.max()
        
    dat = data[(data.time>=start)&(data.time<=end)]
            
    # NEW 1. Sliding window to check condition 
    y = abs(dat[mscl]).reset_index(drop=True)
    w_ms = window
    w_ticks = w_ms * 5

    a = y >= thrs

    s = np.zeros(len(a)-w_ticks+1, dtype=int)

    s[0] = np.sum(a[:w_ticks])

    for i in tqdm(range(1, len(s), 1)):
        s[i] = s[i-1] - a[i-1] + a[i+w_ticks-1]

    cond_result = s >= (w_ticks * positive)
    time = dat.time[:len(dat)-w_ticks+1]
    
      

    # 2. Find "start" and "end" of movement
    result3 = pd.DataFrame({'times':time, 'result':cond_result})
    result3['previous'] = result3.result.shift(1)
    result3['change'] = result3.result - result3.previous

    trig = result3[result3.change.isin([1,-1])].reset_index(drop=True)
    trig['movement'] = np.where(trig.change==1, 'start', 'end')



    # 3. select times in the suitable windows preciselly 
    res = []
    for tr in trig.times:
        wnd = np.abs(dat[(dat.time>=tr)&(dat.time<=tr+window)])

        if trig[trig.times==tr].movement.values[0] == 'start':
            # select time when uV achived thrs first
            first = wnd[wnd[mscl]>=thrs2].time.min()
            res.append([first,trig[trig.times==tr].movement.values[0]])
        else: 
            last = wnd[wnd[mscl]>=thrs2].time.max()
            res.append([last,trig[trig.times==tr].movement.values[0]])
        
    markup = pd.DataFrame(res)
    if markup.shape[1]>1:
        markup.columns = ['time','name']
    else:
        pass
        
    return markup, dat

def plot_signal(data, movement_time, mscl):
    """
    Interactive plot to see movement start closer

    Arguments:
      data: DataFrame with EMG by channels
      movement_time: array with times of movement start/end
    """

    fig, (ax0) = plt.subplots(nrows=1)
    for r in movement_time:
        ax0.axvline(x=r, color='r')

    ax0.plot(data.time, data[mscl], label='signal')
    ax0.legend()
    
    
def plot_emg_events(raw, movement_time):
    """
    MNE Interactive plot to see whole EMG with movement time markups

    Arguments:
      raw: emg file
      movement_time: array with times of movement start/end
    """
    # create event list from movemet_times and name each event with order number
    events = pd.DataFrame({'time':np.array(movement_time)*5, 'event_value':1, 'event_name':list(range(1,len(np.array(movement_time))+1))})
    filt = np.array([50, 100, 150, 200])
    notch = raw.notch_filter(freqs=filt, verbose=False) #filter
    notch.plot(n_channels=1, event_color='red', 
             events=np.array(events), 
             scalings=1e-3, 
             highpass=20, lowpass=1000)
    
def create_events(movement_time):
    """
    Creates event list from movemet_times and name each event with order number

    Arguments:
      movement_time: array with times of movement start/end
    """
    events = pd.DataFrame({'time':np.array(movement_time), 'event_value':1, 'event_name':list(range(1,len(np.array(movement_time))+1))})
    return events

    



def adjust_movements(ev):
    """
    Interactive program based on widgets to Manually adjast Algo movement markups

    Arguments:
      ev: array with time of movement markups which are need to be adjasted
    """
    
    #raw file to dataframe
    data = raw.filter(l_freq=20, h_freq=1000, verbose=False).to_data_frame()
    
    # create widget to list events with button 'next event'
    button=widgets.Button(
        value=False,
        description='Next event',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='Saved' # (FontAwesome names without the `fa-` prefix)
                        )

    
    l = [] # for loop - indexes list of redo events 
    new_ts = [] # list to save new times stamp

    def plus(sender):
        l.append(1)
        index = np.array(l).sum()
        try:
            
            # create windget to Change event position
            w = widgets.FloatText(
            value=ev[index-1],
            min=ev[index-1]-800,
            max=ev[index-1]+3000,
            step=5,
            description='New Value uV:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            )
            
            # function for visualisation of new timestamp based on windget to Change event position
            def f(x):
                fig, (ax0) = plt.subplots(nrows=1)
                ax0.axvline(x=x, color='r')
                ax0.plot(data[(data.time>=ev[index-1]-800)&(data.time<=ev[index-1]+3000)].time, 
                         data[(data.time>=ev[index-1]-800)&(data.time<=ev[index-1]+3000)][mscl], label='signal') 

            
            # create Button to save new position
            b=widgets.Button(
                value=False,
                description='Save timestamp',
                disabled=False,
                button_style='', # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Description',
                icon='Saved' # (FontAwesome names without the `fa-` prefix)
                )
            
            
            saved_ts=[] # new timestamp list
            
            # function of button 'save timestemp' - it saves new value to the list
            def but(sender):
                saved_ts.append(w.value)

            b.on_click(but)
            display(b,interactive(f, x=w))
            new_ts.append(saved_ts)
            
        except Exception as e:
            print("You finished")


    button.on_click(plus)
   

    
    return display(button),new_ts


# 1. SET UP GOOGLE SHEET API
# ----------- CHANGE CREADENTIALS LOCATION ------------------------

# #current_dir = '/content'
# current_dir = key_path
# GSPREAD_KEYS_PATH = current_dir + '/esoteric-pad-326113-4ab241befdd8.json' # this is the directory where your file with key located
 

# # Some functions for authorizing (no need to change something here)
# def authorize_at_gspread(gspread_key_path=GSPREAD_KEYS_PATH):
#     """
#     Return authorized gspread object
#     :param gspread_key_path: path to credentials key
#     :return: gc object
#     """
#     scope = ['https://spreadsheets.google.com/feeds']
#     credentials = ServiceAccountCredentials.from_json_keyfile_name(GSPREAD_KEYS_PATH, scope)
#     gc = gspread.authorize(credentials)
#     return gc
# # authorization
# authorize_at_gspread()


# 2. Function for particular sheet connection
# def get_workbook(workbook_url, gspread_key_path=GSPREAD_KEYS_PATH):
#     """ Returns GSPREAD object with opened workbook.
#     Just open worksheet by referring to .worksheet(name) or use gspread_dataframe to load dataframe.
#     """
#     gc = authorize_at_gspread(gspread_key_path=gspread_key_path)
#     return gc.open_by_url(workbook_url)


# Damir - to write to a sheet fucntion
# def tospreadsheet (df, workbook_link, header = 'Title', key = 'new'):
#     """Import dataframe to google spreadsheet by link
#     Args: 
#         df: imported dataframe
#         workbook: link to dataframe
#         header: header of worksheet 
#         key: default 'new'
#         if you want to use existing worksheet pass 'to existing sheet'

#     """
#     if key == 'new':
#         workbook = get_workbook(workbook_link)
#         wks = workbook.add_worksheet(header, len(df.index),len(df.columns))
#         gsp.set_with_dataframe(wks, df, row = 1, col =1, include_column_header=True)

#     elif key == 'existing':
#         workbook = get_workbook(workbook)
#         wks = workbook.worksheet(header)
#         gsp.set_with_dataframe(wks, df, row = 1, col =1, include_column_header=True)
        

def gsheet(header,new_t,bad,slow, sheet=False):
    
    new = pd.DataFrame({'events_to_correct':redo})
    new2=pd.concat([new, pd.DataFrame(new_t[1])], axis=1).set_index('events_to_correct')

    result = movement_time[['name','time']]
    result['event_name'] = result.reset_index()['index']+1
    result = result.set_index('event_name')

    final = pd.concat([result,new2],axis=1)
    final['muscle'] = mscl
    final['file'] = path[-20:path.find('.vhdr')][path[-20:path.find('.vhdr')].find('/')+1:]
    #final_corrected_movement_time = last filled column before "muscle" column
    final['final_corrected_movement_time'] = final.apply(lambda x: x.iloc[(x.dropna().to_frame().transpose().columns.get_loc('muscle'))-1],axis=1)

    #add Bad and Slow lables
    final['slow'] = pd.DataFrame({'index':slow, 'Slow':True}).set_index('index')
    final['bad'] = pd.DataFrame({'index':bad, 'Bad':True}).set_index('index')
    
    link = 'https://docs.google.com/spreadsheets/d/1g0wJeyONCuivtppNc352MBkBIyOgY7FxkQq5XgaBjsg/edit?usp=sharing'

    if sheet == True:
        pass
        #tospreadsheet(final, link, header)
    elif sheet == False:
        file = header + '.xls'
        final.to_excel(file)
        print('Movement data saved to file ', file)

    return final


#PMA functions

def envelop(time, mscl, data, sub_window_size):
    """
    time: list, time where calculate envelop
    mscl: str
    data: df
    subwindow_size: int, ms for envelop
    """
    
    start = time[0]
    end = time[1]    
    array = np.array(data[mscl])
    
    start_index = data[data.time>=start].index.min()
    end_index = data[data.time<=end].index.max()    
        
        
    sub_windows = (
        start_index + 
        # *5 cos 5k HZ
        np.expand_dims(np.arange(sub_window_size*5), 0) +
        # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
        np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
        )
    
    envelop = pd.DataFrame(array[sub_windows]).apply(lambda x: (abs(x)).max(), axis = 1)
    
    return envelop

def pma_find_preact_thrs(sheet, data, preactivation_time, mscl):
    """
    sheet: df, data about movement from sheet with correected time
    data: df, raw emg data
    preactivation_time: int, is ms for preactivation window analisys
    mscl: str, name of channel
    """
    # extract movement start corrected
    movement_time_cor = sheet[sheet.name=='start'].final_corrected_movement_time
    windows = data[data.time.isin(movement_time_cor)].index
    each_five = list(range(0,len(windows),5)) #select each 1st index among 5 indexes with te same time (5000 hz) 
    # find inndex for movement
    movemenet_start_index = windows[each_five]
    #get trial and its slow lable
    slow_trial_list = sheet[['trial_number','slow']].drop_duplicates().slow
    #save all inffor to df
    preactivation_boards_list = pd.DataFrame({'start_time':np.array(data.time)[movemenet_start_index-preactivation_time*5],
                         'end_time':np.array(data.time)[movemenet_start_index],
                        'Trial number':sheet.trial_number.unique(),
                        'Is slow':slow_trial_list})
    
    thrss = []
    # go through [start, end] times in preactivation
    for preact_time in np.array(preactivation_boards_list):
        env = envelop(preact_time, mscl, data, 50)
        thrs = env.mean() + env.std()*2
        
        # saves thr, original trial number and slow lable
        thrss.append([thrs, preact_time[2], preact_time[3] ])
        
    return pd.DataFrame(thrss)
        
def get_movement_indexes(sheet, name, df):
    """
    sheet: df from sheet with 'final_corrected_movement_time' column
    name: str, time name from sheet start/end of movement
    df: raw.to_data_frame()
    """
    movement_time_cor = sheet[sheet.name==name].final_corrected_movement_time
    windows = df[df.time.isin(movement_time_cor)].index
    each_five = list(range(0,len(windows),5)) #select each 1st index among 5 indexes with te same time (5000 hz) 
    movemenet_index = windows[each_five]
    return movemenet_index

def find_pma(data, mscl, thrss, pma_start_positive):
    
    """
    data: df of emg 
    mscl: str, name of channel like 'lADM' 
    thrss: list, preactivation thresholds list for the mscl
    pma_start_positive: float, share of picks whihch should be >= preact. thrs
    """

    ind_end = get_movement_indexes(sheet, 'end', data)
    ind_st = get_movement_indexes(sheet, 'start', data)

    all_ind = np.stack((ind_st,ind_end), axis=1)

    window_indexes = []
    
    for r in all_ind:
        window_indexes.append(np.array(range(r[0],r[1]+1)))
    
    movement_indexes = np.asarray(window_indexes)

    post_windows = pd.DataFrame()
    
    post_windows['indexes'] = movement_indexes
    post_windows['Trial in session'] = thrss[1]
    post_windows['Is slow'] = thrss[2]
    
    # movement signal
    post_windows['preact_window'] = post_windows.apply(lambda x: np.array(data[mscl][range(x.indexes.min()-5000, x.indexes.min()+1)]), axis = 1)
    post_windows['signal'] = post_windows.apply(lambda x: np.array(data[mscl][x.indexes]), axis=1)
    post_windows['time'] = post_windows.apply(lambda x: np.array(data.time[x.indexes]), axis=1)
    post_windows['preactivation_mean_uv'] = post_windows.apply(lambda row: np.mean(abs(row.preact_window)), axis=1)
    post_windows['preactivation_max_uv'] = post_windows.apply(lambda row: np.max(abs(row.preact_window)), axis=1)
    post_windows['preactivation_std_uv'] = post_windows.apply(lambda row: np.std(abs(row.preact_window)), axis=1)
    post_windows['preactivation_auc_uv'] = post_windows.apply(lambda row: auc(range(len(row.preact_window)),abs(row.preact_window)), axis=1)
        

    post_windows['movement_start'] = post_windows.apply(lambda row: row['time'].min(), axis = 1)
    post_windows['movement_end'] = post_windows.apply(lambda row: row['time'].max(), axis = 1)
    post_windows['preac_thrs'] = thrss[0]
    
    
    def find_start_pma(y, tr, pma_start_positive):
        w = 50 #len of sliding window in ticks (10 ms X 5 hz)
        w_tr = int(pma_start_positive*w) #number of ticks over tr to find pma
        for i in range (len(y)-w):
            y2 = [x for x in y[i:i+w] if abs(x)>tr]
            #y[i:i+w][y[i:i+w]>tr]
            if len(y2) >= w_tr:
                #print(y2,i)
                return i

    def find_end_pma(y, tr, pma_start_positive):
        last_index = find_start_pma(y[::-1], tr, pma_start_positive)
        return len(y)-last_index
        

    # based on preactivation
    post_windows['pma_start_index'] = post_windows.apply(lambda row: find_start_pma(row.signal, row.preac_thrs, pma_start_positive), axis=1)
    post_windows['pma_end_index'] = post_windows.apply(lambda row: pd.DataFrame(row.signal)[abs(row.signal)>row.preac_thrs].index.max()
                                                  , axis=1)
#     post_windows['pma_end_index'] = post_windows.apply(lambda row: find_end_pma(row.signal, row.preac_thrs, pma_start_positive), axis=1)

    post_windows['pma_end_index'] = np.where(post_windows['pma_start_index'].isna()==False, post_windows['pma_end_index'], np.NaN) 
    

    def find_time(array_with_data, index):
        try:
            return int(array_with_data[int(index)])
        except Exception as s:
            pass


    post_windows['pma_time_start'] = post_windows.apply(lambda row: find_time(row.time, row.pma_start_index)
                                                  , axis=1)
    post_windows['pma_time_end'] = post_windows.apply(lambda row: find_time(row.time, row.pma_end_index)
                                                  , axis=1)
    post_windows['pma_duration_ms'] = post_windows['pma_time_end'] - post_windows['pma_time_start']
    
    post_windows['muscle'] = mscl

    
    return post_windows


def check_pma(pma_df, pma_duration):
    """
    pma_df: df recieved from find_pma function
    pma_duration: int, ms how long pma should be
    
    """
    check_pma_df = pma_df
    check_pma_df['pma_check'] = check_pma_df.dropna().apply(lambda x: x.signal[int(x.pma_start_index):int(x.pma_end_index+1)], axis = 1)
    check_pma_df['rms_pma'] = check_pma_df.dropna().apply(lambda x: ((abs(x.pma_check)**2).mean())**0.5, axis = 1)

    def env_pma(line, sub_window_size):
        array = line.signal
        start_index = int(line.pma_start_index)
        end_index = int(line.pma_end_index)   

        sub_windows = (
            start_index + 
            # *5 cos 5k HZ
            np.expand_dims(np.arange(sub_window_size*5), 0) +
            # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
            np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
            )

        envelop = pd.DataFrame(array[sub_windows]).apply(lambda x: (abs(x)).max(), axis = 1)

        return np.array(envelop).mean()

    check_pma_df['pma_envelop_mean'] =  check_pma_df.dropna().apply(lambda x: env_pma(x,pma_duration), axis=1 )

    return check_pma_df


def rms_pma(pma_df):
    """
    pma_df: df recieved from find_pma function
    """
    
    pma_df2 = pma_df.reset_index(drop=True)
    
    # "try" for cases when no pma 
    try:
        pma_df2['pma_wnd'] = pma_df2[['signal','pma_start_index','pma_end_index']].dropna().apply(lambda x: x.signal[int(x.pma_start_index):int(x.pma_end_index+1)], axis = 1)
        pma_df2['pma_mean_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: np.mean(abs(row.pma_wnd)), axis=1)
        pma_df2['pma_max_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: np.max(abs(row.pma_wnd)), axis=1)
        pma_df2['pma_std_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: np.std(abs(row.pma_wnd)), axis=1)
        pma_df2['pma_auc_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: auc(range(len(row.pma_wnd)),abs(row.pma_wnd)), axis=1)
    except:
        pass
    
    def rms(line, sub_window_size):
        array = line
        start_index = 0
        end_index = len(array) - 1  

        sub_windows = (
            start_index + 
            # *5 cos 5k HZ
            np.expand_dims(np.arange(sub_window_size*5), 0) +
            # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
            np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
            )

        #rms_pma = pd.DataFrame(array[sub_windows]).apply(lambda x: ((abs(x)**2).mean())**0.5, axis = 1)
        rms_pma = np.array([((abs(wnd)**2).mean())**0.5 for wnd in array[sub_windows]]).mean()

        return rms_pma
    
    # "try" for cases when no pma 
    try:
        pma_df2['pma_rms_mean'] =  pma_df2[['pma_wnd']].dropna().apply(lambda x: rms(x.pma_wnd, 30), axis=1 )
    except:
        pass
    pma_df2['preactivation_window_rms_mean'] =  pma_df2[['preact_window']].apply(lambda x: rms(x.preact_window, 30), axis=1 )
     
    return pma_df2


def active_hand(all_pma_data, data, rms_time, active_mscl):
    
    """
    
    pma_data: df given from pma function columns indexes, movement_start/end are needed
    data: df, emg data freq. filtered - to get pre/activation data in right hand
    rms_time: ms, window size to calc rms
    active_mscl: str, active muscle
    """
    
    all_pma_data = all_pma_data.reset_index(drop=True)
    active_hand = pd.DataFrame()
    
    active_hand['indexes'] = all_pma_data.indexes
    active_hand['Trial in session'] = all_pma_data['Trial in session']
    active_hand['Is slow'] = all_pma_data['Is slow']
    active_hand['movement_start'] = all_pma_data.movement_start
    active_hand['movement_end'] = all_pma_data.movement_end
    
    def rms(line, sub_window_size):
        array = line
        start_index = 0
        end_index = len(array) - 1  

        sub_windows = (
            start_index + 
            # *5 cos 5k HZ
            np.expand_dims(np.arange(sub_window_size*5), 0) +
            # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
            np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
            )
        #rms_pma = np.array([((abs(wnd)**2).mean())**0.5 for wnd in array[sub_windows]]).mean()
        rms_pma = pd.DataFrame(array[sub_windows]).apply(lambda x: ((abs(x)**2).mean())**0.5, axis = 1).mean()
        return rms_pma

    active_hand_all = pd.DataFrame()
    for m in tqdm(['rAPB','rADM','rECU']): 
        active_hand['preactivation_window'] = active_hand.apply(lambda x: np.array(data[m][range(x.indexes.min()-5000, x.indexes.min()+1)]), axis = 1)
        active_hand['activation_window'] = active_hand.apply(lambda x: np.array(data[m][x.indexes]), axis=1)

        active_hand['preactivation_mean_uv'] = active_hand.apply(lambda row: np.mean(abs(row.preactivation_window)), axis=1)
        active_hand['preactivation_max_uv'] = active_hand.apply(lambda row: np.max(abs(row.preactivation_window)), axis=1)
        active_hand['preactivation_std_uv'] = active_hand.apply(lambda row: np.std(abs(row.preactivation_window)), axis=1)
        active_hand['preactivation_auc_uv'] = active_hand.apply(lambda row: auc(range(len(row.preactivation_window)),abs(row.preactivation_window)), axis=1)
        #active_hand['preactivation_window_baseline_rms_mean'] = active_hand.apply(lambda row: rms(abs(row.preactivation_window),rms_time), axis=1)

        active_hand['activation_mean_uv'] = active_hand.apply(lambda row: np.mean(abs(row.activation_window)), axis=1)
        active_hand['activation_max_uv'] = active_hand.apply(lambda row: np.max(abs(row.activation_window)), axis=1)
        active_hand['activation_std_uv'] = active_hand.apply(lambda row: np.std(abs(row.activation_window)), axis=1)
        active_hand['activation_auc_uv'] = active_hand.apply(lambda row: auc(range(len(row.activation_window)),abs(row.activation_window)), axis=1)
        #active_hand['activation_window_baseline_rms_mean'] = active_hand.apply(lambda row: rms(abs(row.activation_window),rms_time), axis=1)
        active_hand['muscle'] = m
        active_hand_all = active_hand_all.append(active_hand)
        
    active_hand_all['movement_start'] = np.where(active_hand_all['muscle'] == active_mscl, active_hand_all['movement_start'], np.NaN)
    active_hand_all['movement_end'] = np.where(active_hand_all['muscle'] == active_mscl, active_hand_all['movement_end'], np.NaN)
    active_hand_all['active_muscle'] = active_mscl
    
    
    return active_hand_all


# Part 1: find Movement onset

### Uppload data

<p style="color:#FF0000";><b>1. Insert subject .vhdr file PATH !_</p>

In [129]:
path = '/Users/ksenia.kozlova/Documents/GitHub/pma_grant/EMG data/StSh_24/rAPB_70max_1.vhdr'




### LOAD DATA
emg = mne.io.read_raw_brainvision(path,verbose=False)

# FILTERS
# high pass and low pass filters
filters_hp_lp = emg.load_data().filter(l_freq=20, h_freq=1000, verbose=False)
# notch filters
filters_notch = np.array([50, 100, 150, 200])

raw = filters_hp_lp.notch_filter(freqs=filters_notch, verbose=False)
data = raw.to_data_frame()

<p style="color:#FFA500";><b>Information about the record</p>


In [128]:
print(raw.info)

<Info | 7 non-empty values
 bads: []
 ch_names: rAPB, rADM, rECU, lAPB, lADM, lECU
 chs: 6 EEG
 custom_ref_applied: False
 highpass: 20.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2021-07-15 12:42:40 UTC
 nchan: 6
 projs: []
 sfreq: 5000.0 Hz
>


## Plot EMG

<p style="color:#FFA500";><b>Check whether record is ok</p>

In [107]:
try:
    raw.plot(verbose=False)
except:
    print('Error')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Find movement ALGO

__1. Setup AlGO parameters__

1.1. Select trail part of data with 3-5 movements 

<p style="color:#FF0000";><b>Adjast: muscle, start_trial, end_trial </p>

In [112]:
# active muscle channel
muscle = 'rAPB'
# time in ms
start_trial = 0
end_trial = 26000


# no changes
trial_data = data[(data.time>=start_trial)&(data.time<=end_trial)]

Plot Trial signal
<p style="color:#FFA500";><b>check trial data</p>

In [113]:
# no change
fig, (ax0) = plt.subplots(nrows=1)
ax0.plot(trial_data.time, trial_data[str(muscle)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<p style="color:#FF0000";><b>2. Adjast. parameters (thrs, thrs2) !_ </p> 

In [114]:
# ADJUST

# 1. Thresholds, (absolut value) uV activity threshold which concidered as a movement 
# thrs - general, thrs2 is more accurate in subwindow 
thrs = 150
thrs2 = 90

# 2. time window to analyse (ms)
# 650 is optimal 
window = 550

# 3. percentage of picks which should be higher than thrs in the window 
# 0.05 is optimal
positive = 0.05

3. Try algo settings on trial data at first.

In [115]:
# no changes
# start and end in ms
trial_time = [start_trial, end_trial]
# Run algo on trial data
trial_movement_time, trial_data = find_movement(data, trial_time, muscle, window, thrs, thrs2, positive)

trial_movement_time

,time,name
0,11843.0,start
1,14752.0,end
2,16329.0,start
3,19184.0,end
4,21086.0,start
5,25556.0,end


<p style="color:#FFA500";><b>check how algo works</p>

In [116]:
# plot trial
plot_signal(trial_data, trial_movement_time.time, muscle)
#plt.hlines(50, xmin=5000, xmax=22500,color='r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

__If the Movement start is not acurate enough, Adjast thrs,thrs2 and try again #2__

4. Apply to the whole recording (here time = [0] which means "start from 0 second"

In [117]:
last = data.time.max()

In [118]:
movement_time, movement_data = find_movement(data, [0], muscle, window, thrs, thrs2, positive)

<p style="color:#FFA500";><b>check how algo works for whole file</p>

In [119]:
# plot Algo Markup way1
plot_signal(movement_data, movement_time.time, muscle)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Manual adjastment of Algo Movement Markup

<p style="color:#FFA500";><b>check how algo works for whole file</p>

In [120]:
# plot Algo Markup way2
plot_emg_events(raw, movement_time.time)

Setting up band-pass filter from 20 - 1e+03 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 20.00, 1000.00 Hz: -6.02, -6.02 dB



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<p style="color:#FF0000";><b>1. Fill events start numbers to redo/slow/bad !_</p>

In [121]:
## change

#list to manually redo
redo = [22,24,79]
#list with slow (second spike is later than 200 ms)
slow = [25,31,67]

#list with Bad - to delete
bad = [47,49,63,73,75,77]


In [122]:
# no change
events = create_events(movement_time.time)

In [123]:
events[events.event_name.isin(redo)].reset_index().time

0     72645.0
1     79190.0
2    234890.0
Name: time, dtype: float64

__2. Manually adjast movement markup__

In [124]:
mscl = muscle
new_timings = adjust_movements(events[events.event_name.isin(redo)].reset_index().time)

Button(description='Next event', icon='Saved', style=ButtonStyle(), tooltip='Description')

Button(description='Save timestamp', icon='Saved', style=ButtonStyle(), tooltip='Description')

interactive(children=(FloatText(value=72645.0, description='New Value uV:', step=5.0), Output()), _dom_classes…

Button(description='Save timestamp', icon='Saved', style=ButtonStyle(), tooltip='Description')

interactive(children=(FloatText(value=79190.0, description='New Value uV:', step=5.0), Output()), _dom_classes…

Button(description='Save timestamp', icon='Saved', style=ButtonStyle(), tooltip='Description')

interactive(children=(FloatText(value=234890.0, description='New Value uV:', step=5.0), Output()), _dom_classe…

You finished


__Plot signal with final movement start Markups__

<p style="color:#FF0000";><b>Rename sheet name (=file name + subject) !_</p>

__3. Save Movement times to Google sheet/ Excel together with adjasted__

In [125]:
print('Current emg file: ', path)
file_name = input('Movement file name (subject + emg file name): ')

corrected_time = gsheet(file_name , new_timings, bad, slow)

Current emg file:  /Users/ksenia.kozlova/Downloads/Early subjects 5-13/MedVer_11/pma_APB_3.vhdr
Movement file name (subject + emg file name): MedVer_11_pma_APB_3
Movement data saved to file  MedVer_11_pma_APB_3.xls


<p style="color:#FFA500";><b>check final table</p>

In [ ]:
corrected_time

<p style="color:#FFA500";><b>final check how movement was marked up</p>

In [ ]:
plot_emg_events(raw, corrected_time[corrected_time.bad!=True].final_corrected_movement_time)

 # <br><br><br><br>  Part 2. pMA analysis (one session only)

### 0. Get data

<p style="color:#FF0000";><b>Run the cell and fill all forms !_</p>

In [ ]:
# MAC OS = '/' WINDOWS = '\\'
SIGN_PATH = '/'

In [126]:
try:
    subj = input(f'1. Get EMG data (.vhdr) \n\n Continue with file {path} ? \n Y/N: ')
except:
    subj = 'N'


if subj == 'Y':
    try:
        subject_path = path
    except:
        pass     
else:
    subject_path = input('Insert subject .vhdr path: ')

### LOAD DATA
# FILTERS
raw = mne.io.read_raw_brainvision(subject_path, verbose=False)
# high pass and low pass filters
filters_hp_lp = raw.load_data().filter(l_freq=20, h_freq=1000, verbose=False)
# notch filters
filters_notch = np.array([50, 100, 150, 200])
raw_filtered = filters_hp_lp.notch_filter(freqs=filters_notch, verbose=False)
pma_data = raw_filtered.to_data_frame()
 

movement_path = input('2. Gett movement data from Excel \n Insert .xls file path with movement info: ')
sheet = pd.read_excel(movement_path)
sheet['slow'] = np.where((sheet['slow'] == '')|(sheet.slow.isna()==True), sheet['slow'].shift(1), sheet['slow'])
sheet['bad'] = np.where((sheet['bad'] == '')|(sheet.bad.isna()==True), sheet['bad'].shift(1), sheet['bad'])
sheet['trial_number'] = (2+ np.array(range(sheet.shape[0])))//2
sheet = sheet[(sheet.bad!='TRUE')&(sheet.bad!=1)]
print('Your file: ')
    
sheet.head(3)

1. Get EMG data (.vhdr) 

 Continue with file /Users/ksenia.kozlova/Downloads/Early subjects 5-13/MedVer_11/pma_APB_3.vhdr ? 
 Y/N: Y
2. Gett movement data from Excel 
 Insert .xls file path with movement info: MedVer_11_pma_APB_3.xls


FileNotFoundError: [Errno 2] No such file or directory: 'MedVer_11_pma_APB_3.xls'

<p style="color:#FFA500";><b>Check that record and movemet start comply</p>

__Plot EMG and movement start__

In [ ]:
mscl = sheet.muscle[0]
movement_time = sheet.final_corrected_movement_time
try:
    plot_emg_events(raw_filtered, movement_time)
except:
    plot_signal(pma_data, movement_time, mscl)

## PMA processing

### 1. Analyse left hand

<p style="color:#FF0000";><b>can adjast % of picks in pma find algo. Look "#change" row</p>

In [ ]:
pma_all = pd.DataFrame()

PMA_START_POSITIVE = 0.25 


for muscle in tqdm(['lAPB','lADM','lECU']):
    # culculate preactivation thrs in preactivatuon window based on envelop
    thrss = pma_find_preact_thrs(sheet, pma_data, 1000, muscle)
    # find pma times 
    pma = find_pma(pma_data, muscle, thrss, PMA_START_POSITIVE) 
    # calculate rms or each pma
    rms = rms_pma(pma)
    # save all muscles to one df
    pma_all=pma_all.append(rms)
    print(muscle, 'is done')
    
    
pma_all['pma_latency_ms'] = pma_all.pma_time_start - pma_all.movement_start

In [ ]:
pma_all['session'] = subject_path[-6]
loc = len(subject_path)-subject_path[::-1].find(SIGN_PATH)
pma_all['file'] = subject_path[loc:]
pma_all.head(2)

<p style="color:#FFA500";><b>To check pma start by plot</p>

<p style="color:#FF0000";><b>change mscl - muscle for visualization</p>

In [ ]:
mscl = 'lAPB'
markups = ((pma_all[pma_all.muscle==mscl].pma_time_start).append(pma_all[pma_all.muscle==mscl].pma_time_end.append(pma_all[pma_all.muscle==mscl].movement_start.append(pma_all[pma_all.muscle==mscl].movement_end)))).sort_values()

try:
    plot_emg_events(raw_filtered, markups)
except:
    plot_signal(pma_data, markups, mscl)
    

<p style="color:#FFA500";><b>Check pma average numbers</p>

In [ ]:
round(pma_all.groupby(['muscle']).agg({'pma_duration_ms':'mean',
                                 'pma_latency_ms':'mean',
                                 'pma_rms_mean':'mean',
                                 'pma_mean_uv':'mean',
                                 'pma_wnd':'count', # = how many pma found
                                 'movement_start':'count' # = how many movemnts
                                }),1)

In [ ]:
pma_format = round(pma_all[[
       'preactivation_mean_uv',
       'preactivation_max_uv',
       'preactivation_std_uv',
       'preactivation_auc_uv',
       'preactivation_window_rms_mean',
        'preac_thrs','pma_duration_ms', 'pma_latency_ms',
       'pma_mean_uv', 'pma_max_uv', 'pma_std_uv','pma_auc_uv',
       'pma_rms_mean',
        'muscle','Trial in session', 'session', 'file']].set_index(['Trial in session','muscle','session', 'file']).unstack(1),2)

pma_format.head(4)

### 2. Data of right hand

In [ ]:
active_muscle = sheet.muscle[0]
r_hand = active_hand(pma_all[pma_all.muscle=='lAPB'][['movement_start', 'movement_end','indexes','Trial in session','Is slow']], pma_data, 10, active_muscle)

r_hand['session'] = subject_path[-6]
loc = len(subject_path)-subject_path[::-1].find(SIGN_PATH)
r_hand['file'] = subject_path[loc:]

r_hand.head(3)

<p style="color:#FFA500";><b>Check active hand average numbers</p>

In [ ]:
round(r_hand.groupby(['muscle']).agg({'preactivation_mean_uv':'mean',
                                 'activation_mean_uv':'mean',
                                 'activation_auc_uv':'mean',
                                 'movement_start':'count' # = how many movements done
                                }),1)

In [ ]:
r_hand_format = round(r_hand[['movement_start','movement_end',
       'preactivation_mean_uv',
       'preactivation_max_uv',
       'preactivation_std_uv',
       'preactivation_auc_uv',
       'activation_mean_uv',
       'activation_max_uv',
        'activation_auc_uv',
       'activation_std_uv',
       'muscle','Trial in session','session', 'Is slow','file']].set_index(['Trial in session','muscle','session', 'file']).unstack(1),2)

r_hand_format.head(3)

### 3. Merge Data right and left hand

In [ ]:
pma_format.head(2)

In [ ]:
r_hand_format.head(2)

<p style="color:#FFA500";><b>Check final output Right and Left hands </p>

In [ ]:
output = r_hand_format.merge(pma_format, how='left', on = ['Trial in session','file','session'], suffixes = ['_active','_pma'])
output['active_muscle']  = str(r_hand.active_muscle.unique()[0])
output = output.set_index('active_muscle', append=True)
output

__Save output__

In [ ]:
print('Current EMG file for subject:',subject_path)
subject_name = input('Sibject code: ')
file_name = subject_name + '_' + output.reset_index().file[0] +'_s' + output.reset_index().session[0] + '.xls'
output.to_excel(file_name)
print('pma data saved to: ', file_name)

Mudrich: Latency of pMA was subsequently defined automatically during the delay between burst-onset of FDIVol and the time point, at which muscular activity in the contralateral (resting) FDIpMA exceeded a threshold of its own mean baseline activity (1000 ms pre-FDIVol burst onset) + 2 SD for a time window of at least 10 ms